In [1]:
import pandas as pd
import numpy as np
from matrixprofile import *
from matrixprofile.discords import discords
from matplotlib import pyplot as plt
from scipy.io import arff
from binarytree import Node
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pydotplus
from IPython.display import Image
from pandas.plotting import scatter_matrix

In [2]:
def retrieve_all(Ts): #fornita la Ts calcola e restituisce mp, motifs, motifs_distances e discords
    Ts=Ts[0:127] #rimuovo l'attributo "classe"

    dfMP = pd.DataFrame(Ts).astype(float) # genero Dframe per lavorarci su, DA CAPIRE PERCHE SERVE FLOAT
    mp, mpi = matrixProfile.stomp(dfMP[0].values,window_size) #OK STOMP

    #PREPARO TUPLA DA PASSARE ALLA FUN MOTIF (RICHIEDE TUPLA FATTA DA MP E MPI)
    tupla=mp,mpi

    mot, motif_dist  =motifs.motifs(dfMP[0].values,tupla,2)

    #CALCOLO MOTIFS
   # print('Motifs starting position: '+str(mot)+ ' Motifs values (min distances): '+str(motif_dist))
   # print(" ")

    #CALCOLO DISCORDS
    dis= discords(mp,window_size,2)
    #print('Discords starting position: '+str(dis))
    
    tupla=mp,mot,motif_dist,dis
    return tupla

In [14]:
#riceve la lista di coppie dei motifs per ogni record(Ts), e resittuisce lista di valori singoli

def motifsFilter(MotifsList): 
    l2=np.array([])
    for i in range (len(MotifsList['Motif'])): #per ogni entry (per ogni record)
        numMotif=len(MotifsList['Motif'].loc[i])
        #print(numMotif)
        for j in range (numMotif): # per ogni lista di motif
            l1=MotifsList['Motif'].loc[i] #prima lista
            l2=np.append(l2,l1[j][0]) #prendo primo valore di ogni lista 
    
        MotifsList['Motif'].loc[i]=l2
        l2=np.array([]) #svuoto array
    
    return MotifsList
    

In [15]:
#carico dataset in dataFrame
dataset = arff.loadarff('CBF/CBF_TRAIN.arff')
df = pd.DataFrame(dataset[0]) #30 record su matrice da 128 attributi + 'b': classe appartenenza  
#df.info()
df.shape

(30, 129)

In [16]:
window_size=5
#diz={'Motif':[],'Motif-Dist':[],'Discord':[]}
diz={'Motif':[]}
MotifsList = pd.DataFrame(diz)
print(MotifsList)

for i in range(30):
    Ts = np.array(df.iloc[i].values)
   # print('TS ID:' + str(i))
   # print('TS CLASS:' + str(Ts[128]))
    mp,mot,motif_dist,dis = retrieve_all(Ts)
   # print('MP'+str(mp))
    #print("Motifs"+str(mot))
    #print("Motifs Dist"+str(motif_dist))
    #print("Discords"+str(dis))
    #plot_all(Ts,mp,mot,motif_dist,dis,window_size)
   #MotifsList.loc[i]=mot,motif_dist,dis
    MotifsList['Motif'].loc[i]=mot
        
print('Motif estratti')
print(MotifsList['Motif'])

Empty DataFrame
Columns: [Motif]
Index: []
Motif estratti
0              [[9, 93], [26, 85]]
1          [[105, 108], [54, 113]]
2             [[17, 56], [40, 94]]
3       [[48, 61], [12, 109, 115]]
4                       [[27, 95]]
5        [[45, 60], [13, 17, 122]]
6              [[7, 48], [37, 43]]
7              [[2, 92], [24, 56]]
8              [[1, 11], [29, 52]]
9             [[50, 103], [0, 33]]
10            [[11, 26], [17, 75]]
11             [[31, 55], [7, 60]]
12                     [[53, 105]]
13            [[79, 84], [6, 106]]
14                      [[59, 70]]
15             [[2, 91], [45, 59]]
16    [[28, 44, 86, 117], [3, 34]]
17           [[14, 20], [35, 117]]
18             [[13, 71], [3, 48]]
19             [[19, 55], [6, 67]]
20            [[19, 78], [39, 85]]
21                      [[26, 84]]
22            [[18, 28], [22, 65]]
23           [[27, 116], [42, 63]]
24         [[8, 74, 87], [28, 65]]
25                      [[16, 81]]
26            [[67, 78], [59, 86

In [17]:
MotifsList=motifsFilter(MotifsList)
print(MotifsList['Motif'])
Ts = np.array(df.iloc[0].values)
print(Ts[9:(9+5)])
print(Ts[26:(26+5)])

0       [9.0, 26.0]
1     [105.0, 54.0]
2      [17.0, 40.0]
3      [48.0, 12.0]
4            [27.0]
5      [45.0, 13.0]
6       [7.0, 37.0]
7       [2.0, 24.0]
8       [1.0, 29.0]
9       [50.0, 0.0]
10     [11.0, 17.0]
11      [31.0, 7.0]
12           [53.0]
13      [79.0, 6.0]
14           [59.0]
15      [2.0, 45.0]
16      [28.0, 3.0]
17     [14.0, 35.0]
18      [13.0, 3.0]
19      [19.0, 6.0]
20     [19.0, 39.0]
21           [26.0]
22     [18.0, 22.0]
23     [27.0, 42.0]
24      [8.0, 28.0]
25           [16.0]
26     [67.0, 59.0]
27     [16.0, 39.0]
28     [11.0, 41.0]
29      [6.0, 15.0]
Name: Motif, dtype: object
[-0.91830825 -0.92210226 -0.98448828 -1.2880511 -1.14346]
[-1.2977275 1.3314643 0.9357379 1.2360468 1.0560677]


In [96]:
#Fun MOTIFS Returns tuple (motifs, distances)
    #motifs: a list of lists of indexes representing the motif starting locations.
    #distances: list of minimum distances for each motif
#riceve la lista degli indici di partenza dei motif, e le Ts, per ognuno di essi(motifs) genera un DistanceProfile 

def retrieveDistanceProfile(df,MotifsList):
    diz={'Index':[],'TsIndex':[],'motifStartingIndex':[],'Used':[],'SubSequenceDistArray':[]}
    dfSubSeqDist=pd.DataFrame(columns=['Index','TsIndex','motifStartingIndex','Used','SubSequenceDistArray']) 
    
    
    #dfSubSeqDist -> TsIndex= indice della Ts a cui appartiene il candidato shapelet
    #                motifStartingIndex= indice di posizione di partenza del candidato shapelet nella TS
    #                Used= se è gia stato usato o meno nel DTree
    #                SubSequenceDistArray= array in cui ogni valore in posizione i
    #                                      è la distanza minima tra il candidato e la Ts di indice i
    
    # NB: la coppia TsIndex e motifStartingIndex IDENTIFICANO UNIVOCAMENTE un candidato Shapelet
    # L'INDICE DELLA Ts SI RIFERISCE ALLA POSIZIONE CHE OCCUPA NELLA MOTIFS LIST               
    
    
    counter=0
    
    for i in range(len(MotifsList['Motif'])):
        numMotif=len(MotifsList['Motif'].loc[i]) #quanti starting index di motif ci sono in pos i
        for j in range (numMotif):
            l1=MotifsList['Motif'].loc[i] #lista di indice i in MotifsList
            startingIndex=l1[j] #indice di inizio del motif
            print(startingIndex)
            TsContainingCandidateShapelet = np.array(df.iloc[i].values) #Ts contenente candidato shapelet
            #subSeq=TsSubSeq[int(startingIndex):int(startingIndex)+window_size] #sottosequenza di Ts contenente il motif
            #print(subSeq)
            #print(i,j)
            minValueList = np.array([]) #array contenente le subSeqDist calcolate dalla data SubSeq con tutte le ts
            for k in range (len(df)):
                TsToCompare = np.array(df.iloc[k].values)
                TsToCompare=TsToCompare[0:127]
                Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                #print(Dp[0])
                #print(min(Dp[0]))
                minValueList=np.append(minValueList,min(Dp[0])) #Dp[0] contiene il Dp effettivo
            dfSubSeqDist['Index'].loc[counter]=counter    
            dfSubSeqDist['TsIndex'].loc[counter]=i
            dfSubSeqDist['motifStartingIndex'].loc[counter]=startingIndex
            dfSubSeqDist['Used'].loc[counter]=False
            dfSubSeqDist['SubSequenceDistArray'].loc[counter]=minValueList
            
            counter=counter+1
    
    #print(dfSubSeqDist['SubSequenceDist'])
    return dfSubSeqDist

In [97]:
dfSubSeqDist=pd.DataFrame()
dfSubSeqDist=retrieveDistanceProfile(df,MotifsList)
dfSubSeqDist.set_index('Index')


print(dfSubSeqDist['Index'].iloc[[0]])
print(dfSubSeqDist['TsIndex'].iloc[[0]])
print(dfSubSeqDist['motifStartingIndex'].loc[[0]])
print(dfSubSeqDist['Used'].loc[[0]])
print(dfSubSeqDist['SubSequenceDistArray'].loc[[0]])



print('')
print(len(dfSubSeqDist['Index']))
print(len(dfSubSeqDist['TsIndex']))
print(len(dfSubSeqDist['motifStartingIndex']))
print(len(dfSubSeqDist['Used']))
print(len(dfSubSeqDist['SubSequenceDistArray']))



#quindi ho preso 6 motifs (le prime 3 coppie dalla lista motifs), per ogni startingIndex dei motifs
#ho calcolato Dprofile con tutte le Ts.
#per ogni Dprofile prendo il valore minimo e lo memorizzo in minValueList


9.0
26.0
105.0
54.0
17.0
40.0
48.0
12.0
27.0
45.0
13.0
7.0
37.0
2.0
24.0
1.0
29.0
50.0
0.0
11.0
17.0
31.0
7.0
53.0
79.0
6.0
59.0
2.0
45.0
28.0
3.0
14.0
35.0
13.0
3.0
19.0
6.0
19.0
39.0
26.0
18.0
22.0
27.0
42.0
8.0
28.0
16.0
67.0
59.0
16.0
39.0
11.0
41.0
6.0
15.0
0    0
Name: Index, dtype: int64
0    0
Name: TsIndex, dtype: int64
0    9.0
Name: motifStartingIndex, dtype: float64
0    False
Name: Used, dtype: bool
0    [4.5320112483935206e-07, 0.9839967565923585, 0...
Name: SubSequenceDistArray, dtype: object

55
55
55
55
55


In [ ]:

#TRICK= list(set(row[-1] for row in dataset)) -1 stands for last

